In [47]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import pickle

In [4]:
# Read the Excel file into a DataFrame
df = pd.read_excel('tender_postcode.xlsx') 

# Extract the columns we need
df = df[['Client Agency', 'Reference Number', 'Type of Work', 'Contract Title', 'Description', 'UNSPSC Title', 'Supplier Name', 'Supplier ABN']]

In [6]:
def remove_html_tags(text):
    return BeautifulSoup(text, "html.parser").get_text()

In [7]:
# remove the html format
df['Description'] = df['Description'].apply(remove_html_tags)


In [10]:
unique_suppliers_count = df['Supplier Name'].nunique()
unique_unspc_title = df['UNSPSC Title'].nunique()
print(f"There are {unique_suppliers_count} unique supplier names.")
print(f"There are {unique_unspc_title} unique UNSPSC Title.")


There are 6693 unique supplier names.
There are 2336 unique UNSPSC Title.


In [21]:
df['merged_title'] = df['Contract Title'] + ' ' + ['UNSPSC Title']

In [30]:
df_title = df[[ 'Reference Number',  'merged_title']]

In [35]:
df_title = df_title.rename(columns={
    'Reference Number': 'ref_num'
})


,ref_num,merged_title
0,ARA201901891,supply and delivery of aspen bedding material ...
1,20221AGWA,bar consultancy and staffing agwa rooftop bar ...
2,FIN873DLGSCAG,investment services for art gallery wa unspsc ...
3,2020153WAM,provision of audience research unspsc title
4,CUAHRS202117042023AC,consultancy services for hr support unspsc title
...,...,...
26470,WCT0119,provision of electricity services for workcove...
26471,WCV0119,redevelop workcover wa conference website in w...
26472,WCQ0219,core business systems technology refresh unsp...
26473,WCQ0219,core business systems technology refresh unsp...


In [36]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text


In [37]:
# Let's take a look at the updated text
df_title['merged_title'] = df_title['merged_title'].apply(clean_text)
df_title

,ref_num,merged_title
0,ARA201901891,supply and delivery of aspen bedding material ...
1,20221AGWA,bar consultancy and staffing agwa rooftop bar ...
2,FIN873DLGSCAG,investment services for art gallery wa unspsc ...
3,2020153WAM,provision of audience research unspsc title
4,CUAHRS202117042023AC,consultancy services for hr support unspsc title
...,...,...
26470,WCT0119,provision of electricity services for workcove...
26471,WCV0119,redevelop workcover wa conference website in w...
26472,WCQ0219,core business systems technology refresh unsp...
26473,WCQ0219,core business systems technology refresh unsp...


In [50]:
# Let's pickle it for later use
df_title.to_pickle("corpus.pkl")

In [41]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df_title.merged_title)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_dtm.index = df_title.ref_num
data_dtm

,aasb,aat,aats,ab,abalone,abandoned,abattoir,abbott,abbottst,aberdeen,...,zk,zo,zone,zones,zoo,zoological,zoology,zoom,zscaler,zuytdorp
ref_num,,,,,,,,,,,,,,,,,,,,,
ARA201901891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20221AGWA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FIN873DLGSCAG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020153WAM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CUAHRS202117042023AC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WCT0119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WCV0119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WCQ0219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [48]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
df_title.to_pickle('df_title.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

In [45]:
data = pd.read_pickle('dtm.pkl')
data = data.transpose()
data.head(500)

ref_num,ARA201901891,20221AGWA,FIN873DLGSCAG,2020153WAM,CUAHRS202117042023AC,RFQ13042023ACT,E03052023ACT,RFQ06122022ACT2,CUAAFA201807032023AC,CUAPCS201829112022AC,...,WCQ0220,WCQ0220,WCQ0220,WCQ0120,WCQ0122,WCT0119,WCV0119,WCQ0219,WCQ0219,WCE0119
aasb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aats,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abalone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
arboricultural,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arborist,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arcgis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# Find the top 5 words of each tender
top_dict = {}
for r in data.columns:
    column_data = data.loc[:, r]
    
    # Check if column_data is a DataFrame
    if isinstance(column_data, pd.DataFrame):
        column_data = column_data.iloc[:, 0]  # Take the first column to get a Series
    
    top = column_data.sort_values(ascending=False).head(10)
    top_dict[r] = list(zip(top.index, top.values))

top_dict


{'ARA201901891': [('material', 1),
  ('delivery', 1),
  ('supply', 1),
  ('aspen', 1),
  ('bedding', 1),
  ('unspsc', 1),
  ('title', 1),
  ('plastic', 0),
  ('plastics', 0),
  ('plasticware', 0)],
 '20221AGWA': [('bar', 3),
  ('agwa', 2),
  ('title', 1),
  ('café', 1),
  ('staffing', 1),
  ('consultancy', 1),
  ('rooftop', 1),
  ('unspsc', 1),
  ('aasb', 0),
  ('plasticware', 0)],
 'FIN873DLGSCAG': [('wa', 1),
  ('gallery', 1),
  ('art', 1),
  ('investment', 1),
  ('services', 1),
  ('unspsc', 1),
  ('title', 1),
  ('platfrom', 0),
  ('plastics', 0),
  ('plasticware', 0)],
 '2020153WAM': [('provision', 1),
  ('title', 1),
  ('audience', 1),
  ('unspsc', 1),
  ('research', 1),
  ('plastic', 0),
  ('plastics', 0),
  ('plasticware', 0),
  ('plate', 0),
  ('plates', 0)],
 'CUAHRS202117042023AC': [('unspsc', 1),
  ('services', 1),
  ('consultancy', 1),
  ('hr', 1),
  ('title', 1),
  ('support', 1),
  ('platform', 0),
  ('plasterglass', 0),
  ('plastic', 0),
  ('plastics', 0)],
 'RFQ1304202